In [1]:
from estnltk import Annotation, Span, EnvelopingSpan, Layer, Text, Tagger

# Low-level layer operations
## Create a layer
```python
Layer(name: str,
      attributes: Sequence[str] = (),
      text_object: Text = None,
      parent: str = None,
      enveloping: str = None,
      ambiguous: bool = False,
      default_values: dict = None
      )
```
**name**: the name of the layer, required argument<br/>
**attributes**: the list of the attributes of the layer<br/>
**text_object**: the `Text` object of the layer<br/>
**parent**: the name of the parent layer<br/>
**enveloping**: the layer is enveloping<br/>
**ambiguous**: the layer is ambiguous<br/>
**defauld_values**: default values of the attributes

If `parent is not None`, then `enveloping is None`.<br>
If `enveloping is not None`, then `parent is None`.

### Has no parent, is not enveloping, is  not ambiguous

In [2]:
text = Text('Tere, maailm!')

layer = Layer(name='test_layer',
              text_object=text,
              attributes=['attr_1', 'attr_2']
              )

layer.add_annotation(( 0,  4), attr_1='TERE', attr_2='lambda a: a.layer[1]')
layer.add_annotation(( 6, 12), attr_1='MAAILM', attr_2='lambda a: 12+88')
layer.add_annotation((12, 13), attr_1='!')

layer

Layer(name='test_layer', attributes=('attr_1', 'attr_2'), spans=SL[Span(start=0, end=4, text='Tere'),
Span(start=6, end=12, text='maailm'),
Span(start=12, end=13, text='!')])

### Has no parent, is not enveloping, is ambiguous
The code below differs from the code above by one line:
```python
ambiguous=True
```
Note that in this case equal spans are grouped together.

In [3]:
text = Text('Tere, maailm!')

layer = Layer(name='test_layer',
              text_object=text,
              attributes=['attr_1'],
              ambiguous=True
              )
layer.add_annotation((0,  4), attr_1='TERE')
layer.add_annotation((6, 12), attr_1='MAAILM')
layer.add_annotation((6, 12), attr_1='MaaIlm')

layer

Layer(name='test_layer', attributes=('attr_1',), spans=SL[AS(start=0, end=4, text:'Tere'),
AS(start=6, end=12, text:'maailm')])

### Has a parent, is not enveloping, is  ambiguous

In [4]:
text = Text('Tere, maailm!').tag_layer(['words'])

layer = Layer(name='test_layer',
              text_object=text,
              attributes=['a'],
              parent='words', 
              ambiguous=True
              )
for word in text.words:
    layer.add_annotation(word.base_span, a=1)
    # or the same in 3 lines
    span = Span(base_span=word.base_span, parent=word, layer=layer)
    span.add_annotation(Annotation(span, a=2))
    layer.add_span(span)

layer

Layer(name='test_layer', attributes=('a',), spans=SL[AS(start=0, end=4, text:'Tere'),
AS(start=4, end=5, text:','),
AS(start=6, end=12, text:'maailm'),
AS(start=12, end=13, text:'!')])

### Does not have a parent, is enveloping, is not  ambiguous

In [5]:
t = 'Kolmkümmend kolm on suurem kui kaheksateist.'
text = Text(t)
text.tag_layer(['words'])

layer = Layer(name='test_layer',
              text_object=text,
              enveloping='words',
              attributes=['value'],
              ambiguous=False)

spl = EnvelopingSpan(spans=text.words[0:2], layer=layer)
spl.value = '33'
layer.add_span(spl)

spl = EnvelopingSpan(spans=[text.words[1], text.words[4]], layer=layer)
spl.value = 0
layer.add_span(spl)

spl = EnvelopingSpan(spans=text.words[5:6], layer=layer)
spl.value = '18'
layer.add_span(spl)

layer

Layer(name='test_layer', attributes=('value',), spans=SL[ES[Span(start=0, end=11, text='Kolmkümmend'),
Span(start=12, end=16, text='kolm')],
ES[Span(start=12, end=16, text='kolm'),
Span(start=27, end=30, text='kui')],
ES[Span(start=31, end=43, text='kaheksateist')]])

The next demo phrase tagger marks all consequtive uppercase words inside a sentence.

In [6]:
import itertools

class UppercasePhraseTagger(Tagger):
    """Tag uppercase phrases

    """
    conf_param = []
    
    def __init__(self):
        self.input_layers=['words', 'sentences']
        self.output_layer='uppercasephrase'
        self.output_attributes=['phrasetext', 'tag']
    
    def _make_layer(self, text: Text, layers, status) -> Layer:
        layer = Layer(name=self.output_layer, 
                      text_object=text, 
                      enveloping='words', 
                      attributes=self.output_attributes)

        uppercases = []
        prevstart = 0
        for sentence in text.sentences:
            for idx, word in enumerate(sentence.words):
                if word.text.upper() == word.text and word.text.lower() != word.text:
                    uppercases.append((idx + prevstart, word))
            prevstart += len(sentence)

        from operator import itemgetter
        from itertools import groupby
        rs = []
        for k, g in groupby(enumerate(uppercases), lambda i: i[0] - i[1][0]):
            r = map(itemgetter(1), g)
            rs.append(list(r))

        spans = [[j for _, j in i] for i in rs if len(i) > 1]

        for idx, s in enumerate(spans):
            layer.add_annotation(s, phrasetext = ' '.join([i.text for i in s]).lower(), tag=idx)

        return layer


tagger = UppercasePhraseTagger()
text = Text('Minu KARU ON PUNANE. MIS värvi SINU KARU on? Kuidas PALUN?')
text.tag_layer(['words', 'sentences'])
tagger.tag(text)

Text(text='Minu KARU ON PUNANE. MIS värvi SINU KARU on? Kuidas PALUN?')

In [7]:
text.uppercasephrase

Layer(name='uppercasephrase', attributes=('phrasetext', 'tag'), spans=SL[ES[Span(start=5, end=9, text='KARU'),
Span(start=10, end=12, text='ON'),
Span(start=13, end=19, text='PUNANE')],
ES[Span(start=31, end=35, text='SINU'),
Span(start=36, end=40, text='KARU')]])

In [8]:
text.tag_layer(['morph_analysis'])
print(text.uppercasephrase.get_attributes(['phrasetext', 'text']))
#[[('karu on punane', 'KARU'), ('karu on punane', 'ON'), ('karu on punane', 'PUNANE')], [('sinu karu', 'SINU'), ('sinu karu', 'KARU')]]

print(text.phrasetext)
#['karu on punane', 'sinu karu']

print(text.uppercasephrase.lemma)
# [[['karu'], ['olema', 'olema'], ['punane']], [['sina'], ['karu']]]

print([i.text for i in text.words if i not in list(itertools.chain(*text.uppercasephrase.spans))])
# ['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']

mapping ={i:[j for j in text.uppercasephrase.spans if i in j][0] 
          for i in list(itertools.chain(*text.uppercasephrase.spans))}
print([i.text for i in text.words if i not in list(itertools.chain(*text.uppercasephrase.spans))])
# ['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']

print([i.text if i not in mapping.keys() else mapping[i].tag for i in text.words])
# ['Minu', 0, 0, 0, '.', 'MIS', 'värvi', 1, 1, 'on', '?', 'Kuidas', 'PALUN', '?']

[[('karu on punane', 'KARU'), ('karu on punane', 'ON'), ('karu on punane', 'PUNANE')], [('sinu karu', 'SINU'), ('sinu karu', 'KARU')]]
['karu on punane', 'sinu karu']
[[AttributeList(['karu'], ('lemma',)), AttributeList(['olema', 'olema'], ('lemma',)), AttributeList(['punane'], ('lemma',))], [AttributeList(['sina'], ('lemma',)), AttributeList(['karu'], ('lemma',))]]
['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']
['Minu', '.', 'MIS', 'värvi', 'on', '?', 'Kuidas', 'PALUN', '?']
['Minu', 0, 0, 0, '.', 'MIS', 'värvi', 1, 1, 'on', '?', 'Kuidas', 'PALUN', '?']


# Rewriting

Rewriter is a class that contains a `rewrite` method. This method is used to convert the input layer to the resulting layer span by span. If the layer is not ambiguous, the span data is passed as a dict, and if the layer is ambiguous, the span data is passed as a list of dicts. If `rewrite` returns `None`, the corresponding span is not created.

Rewriting a layer `L` creates a new layer `N` such that the parent of `N` is `L`.

In the next example 'test_layer' contains spans with uppercase values in the 'upper' attribute for all but the first word.

In [9]:
class TestRewriter:
    def rewrite(self, record):
        if record['start'] == 0:
            return None
        record['upper'] = record['text'].upper()
        return record

t = Text('Tere maailm!')
t.tag_layer(['words'])
test_layer = t['words'].rewrite(source_attributes=('text',),
                                target_attributes=('upper',),
                                rules = TestRewriter(),
                                name='test_layer')
t['test_layer'] = test_layer
t['test_layer']

Layer(name='test_layer', attributes=('upper',), spans=SL[Span(start=5, end=11, text='maailm'),
Span(start=11, end=12, text='!')])

The previous example demonstrates the case when the rewritten layer and the resulting layer are not ambiguous.

In the next example an ambiguous layer is rewritten and the result is also an ambiguous layer. The demo rewriter makes a copy of the first analysis of every word, reverses the value of `lemma` and adds the modified analysis to the span. Note that the last span does not get the extra annotation because it would be equal to the existing one.

In [10]:
class AmbiguousLayerRewriter:
    def rewrite(self, records):
        records.append(records[0].copy())
        records[-1]['lemma'] = records[-1]['lemma'][::-1]
        return records

t = Text('Tere maailm!').analyse('morphology')
input_layer = t['morph_analysis']
t['demo'] = input_layer.rewrite(source_attributes=input_layer.attributes,
                                target_attributes=input_layer.attributes,
                                rules=AmbiguousLayerRewriter(),                                       
                                name='demo', 
                                ambiguous=True)
t['demo']

Layer(name='demo', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[AS(start=0, end=4, text:'Tere'),
AS(start=5, end=11, text:'maailm'),
AS(start=11, end=12, text:'!')])

The rewriting mechanism does not work with enveloping layers. Here is an example how to create a new layer on top of enveloping layer.

In [11]:
text = Text('Esimene lause? Teine lause! Kolmas lause.').analyse('segmentation')
sentence_types={'.':'statement', '?':'question', '!':'exclamation'}
layer = Layer(name='sentence_type', parent='sentences', attributes=('type',))
for s in text['sentences']:
    s.type = sentence_types.get(s[-1].text)
    layer.add_span(s)

text['sentence_type'] = layer
text['sentence_type']

Layer(name='sentence_type', attributes=('type',), spans=SL[ES[Span(start=0, end=7, text='Esimene'),
Span(start=8, end=13, text='lause'),
Span(start=13, end=14, text='?')],
ES[Span(start=15, end=20, text='Teine'),
Span(start=21, end=26, text='lause'),
Span(start=26, end=27, text='!')],
ES[Span(start=28, end=34, text='Kolmas'),
Span(start=35, end=40, text='lause'),
Span(start=40, end=41, text='.')]])

# Attribute lists
Immutable lists for layer attribute value representation.

In [12]:
from estnltk.layer import AttributeList
from estnltk.layer import AmbiguousAttributeList
from estnltk.layer import AttributeTupleList
from estnltk.layer import AmbiguousAttributeTupleList

In [13]:
al = AttributeList([1,2,3,4], 'attr')
al

,attr
0,1
1,2
2,3
3,4


In [14]:
aal = AmbiguousAttributeList([[1,2], [3,4], [5]], 'attr')
aal

,attr
0,1
,2
1,3
,4
2,5


In [15]:
atl = AttributeTupleList([[1,2,3], [4,5,6], [7,8,9]], ['attr_1', 'attr_2', 'attr_3'])
atl.index = False
atl

,attr_1,attr_2,attr_3
0,1,2,3
1,4,5,6
2,7,8,9


In [16]:
aatl = AmbiguousAttributeTupleList([[[1,2], [3,4]], [[5,6], [7,8], [9,10]], [[11,12]]],
                                   ['attr_1', 'attr_2'])
aatl

,attr_1,attr_2
0,1,2
,3,4
1,5,6
,7,8
,9,10
2,11,12
